In [1]:
import geopandas as gpd
import pandas as pd 
import glob
import os


In [2]:
csv_files = glob.glob(rf'D:\Datasets\Illegal Fishing\Original Data\Ship data\*.csv')

df_list = []
# Loop through each file and add the shiptype column
for file in csv_files:

    df = pd.read_csv(file)
    shiptype = os.path.basename(file).replace('.csv', '')

     # Drop rows where is_fishing = -1
    df = df[df['is_fishing'] != -1]

    df['shiptype'] = shiptype
    df_list.append(df)

ship_data = pd.concat(df_list, ignore_index=True)

ship_data['timestamp'] = pd.to_datetime(ship_data['timestamp'], unit='s')
if 'Unnamed: 0' in ship_data.columns:
    ship_data.drop(columns=['Unnamed: 0'], inplace=True)


In [7]:
# Define the custom function for mapping ranges
def map_range(x):
    if 0 <= x < 0.5 :
        return 0
    elif 0.5 <= x <= 1:
        return 1
     
ship_data['is_fishing'] = ship_data['is_fishing'].apply(map_range)

# Define the mapping
gear_type_mapping = {
    'drifting_longlines': 0,
    'fixed_gear': 1,
    'pole_and_line': 2,
    'purse_seines': 3,
    'trawlers': 4,
    'trollers': 5,
    'unknown': 6
}


ship_data['gear_type_encoded'] = ship_data['shiptype'].map(gear_type_mapping)

ship_data.to_csv(rf'D:\Datasets\Illegal Fishing\Processed Data\Complete_ship_data.csv',index=False)